In [1]:
%load_ext autoreload
%autoreload 2



In [2]:
import math
from typing import List, Tuple, Callable

from app.models.race import Race
from app.models.race_entry import RaceEntry
import hyperopt
from app.db.session import SessionLocal
import random
from pydantic import BaseModel
from decimal import Decimal


In [3]:
db = SessionLocal()

In [11]:
races = db.query(Race).all()
race = races[random.randint(0, len(races)-1)]

print(sum([1/entry.latest_odds() for entry in race.active_entries()]))
entries_sorted = race.entries.copy()
entries_sorted.sort(key=lambda x: x.latest_odds())
print(entries_sorted[0])
print(race.win_pool_total, race.place_pool_total, race.show_pool_total)

1.0000000000000002
<RaceEntry(id=232384,name='Loop Tomb',program_no='4',win_pool_total=665329.4805194805,place_pool_total=599624.4047619047,show_pool_total=651249.8214285714,odds=3.744186046511628)>
3173186.406114719 3041910.346590909 3277845.2053571427


In [12]:
from app.raceday.bet_strategy.bet_strategies import PlaceBet, ShowBet, DrZPlaceShowArbBet, BetStrategy, FlatBetOutlayStrategy, AvgCostRewardSortStrategy

In [13]:
DefaultBetStrategy = BetStrategy(
    outlay_strategy=FlatBetOutlayStrategy(), sort_strategy=AvgCostRewardSortStrategy(),
)

In [14]:
pb = PlaceBet(race=race, entries=race.active_entries(), selection=[entries_sorted[0]], strategy=DefaultBetStrategy)

In [15]:
from app.raceday.bet_strategy.bet_strategies import calc_avg_place_reward, calc_place_reward_redux, calc_place_reward, calc_show_reward, calc_avg_show_reward

In [16]:
drzb = DrZPlaceShowArbBet(race=race, entries=race.active_entries(), selection=[race.active_entries()], strategy=DefaultBetStrategy)

GEN
0.8193549127276736
0.9800107035465426
0.7939447652254894
0.8570277314853867
0.9802580983560851
0.8899914205472057
0.9050342269985316
1.055862382527636
0.9135934213269743
0.9324111401424804
1.0216439795884307
1.2177993559016127
0.7916682907761708
0.7125551298645236


In [17]:
drzb.avg_reward()

3.295305718017679

In [18]:
drzb.max_reward()

38.049517656356024

In [20]:
drzb.cost()

6